In [1]:
from langchain.tools import tool , BaseTool , StructuredTool
from langchain.agents import AgentExecutor , initialize_agent
from langchain.memory import ConversationBufferMemory
from langchain.agents.agent_types import AgentType
import subprocess as sp


In [2]:
from dotenv import load_dotenv
from langchain_google_genai.llms import GoogleGenerativeAI

load_dotenv()

llm = GoogleGenerativeAI(model="gemini-2.0-flash")

In [4]:
import os
curr = os.getcwd()
CODE_PATH = os.path.join(curr , "code")
CODE_PATH

'c:\\Users\\hp\\Desktop\\python practice\\Explore\\day-11\\code'

In [20]:
@tool
def generate_code( input : str) -> str:
    """Generating code"""
    response = llm( f"""
Write a code snippet in python for the given problem.
***OUTPUT ONLY THE CODE WITH NOTHING ELSE***
Problem:
{input}
""")
    if "```python" in response:
        response = response[9:-3]
    with open(os.path.join(CODE_PATH , "temp.py") , "w") as f:
        f.write(response)
    output = sp.run(["python" , os.path.join(CODE_PATH , "temp.py")] , capture_output=True ,text=True , timeout=20 )
    return response if output.returncode==0 else output.stderr

@tool
def test_code( input : str) -> str:
    """Testing code"""
    content = ""
    with open(os.path.join(CODE_PATH , "temp.py") , "r") as f:
        content = f.read()

    response = llm(f"""
"Write a code snippet to execute the given codes with a dummy input.
--(Assume you import the function from temp.py)--.
***OUTPUT JUST CODE SNIPPET AND NOTHING ELSE***,
Code:
{content}
Avoid Below Lines:
{input}
""")
    if "```python" in response:
        response = response[9:-3]
    with open(os.path.join(CODE_PATH , "temp_test.py") , "w") as f:
        f.write(response)
    output = sp.run(["python" , os.path.join(CODE_PATH , "temp_test.py")] , capture_output = True , text = True , timeout = 20)
    return output.stdout if output.returncode==0 else output.stderr



In [21]:
tools = [generate_code , test_code]

memory = ConversationBufferMemory()
agent_chain = initialize_agent(
    tools=tools,
    llm=llm,
    verbose = True,
    memory=  memory
)

In [22]:
agent_chain.invoke({"input":"Write a code to perform quick sort"})



> Entering new AgentExecutor chain...
I need to write a function that implements the quicksort algorithm. Quicksort is a divide-and-conquer algorithm that works by selecting a 'pivot' element from the array and partitioning the other elements into two sub-arrays, according to whether they are less than or greater than the pivot. The sub-arrays are then recursively sorted. I'll start by writing the function definition and then implement the partitioning logic.

Action: generate_code
Action Input: ```python
def quicksort(arr):
    """
    Sorts an array using the quicksort algorithm.

    Args:
        arr: The array to sort.

    Returns:
        The sorted array.
    """
    if len(arr) < 2:
        return arr  # Base case: already sorted

    pivot = arr[0]  # Choose the first element as the pivot
    less = [i for i in arr[1:] if i <= pivot]
    greater = [i for i in arr[1:] if i > pivot]

    return quicksort(less) + [pivot] + quicksort(greater)
```
Observation: 
def quicksort(arr

{'input': 'Write a code to perform quick sort',
 'history': '',
 'output': '```python\ndef quicksort(arr):\n    """\n    Sorts an array using the quicksort algorithm.\n\n    Args:\n        arr: The array to sort.\n\n    Returns:\n        The sorted array.\n    """\n    if len(arr) < 2:\n        return arr  # Base case: already sorted\n\n    pivot = arr[0]  # Choose the first element as the pivot\n    less = [i for i in arr[1:] if i <= pivot]\n    greater = [i for i in arr[1:] if i > pivot]\n\n    return quicksort(less) + [pivot] + quicksort(greater)\n```'}